# Overlaying images on a base map

Maps of areas have been produced for many years, and for a wide range of purposes. The National Archives holds a wide number of these maps; when working with these, it may be useful to overlay the historical maps on a modern base map to provide context. This notebook shows how Folium can be used to overlay several different maps on a base map. 

In this example we are going to progressively overlay a series of maps over the base map of Turkey. This notebook uses two sources from Discovery and one from Wikimedia. If you do not have access to Discovery, you can skip the relevant cells and use only the Wikimedia map. 

## Discovery data

The code in this notebook assumes: 
- "image a.png" is an extract from the map from the last page of [CAB 42/2/8  The future settlement of Eastern Turkey in Asia and Arabia.](https://discovery.nationalarchives.gov.uk/details/r/C6268090), comprising of the top 3 rows, stopping at 34°N.
- "image b.png" is an extract from the map from page 4 of [CAB 24/72/7 Record Type: Memorandum Former Reference: GT 6506A Title: Maps...](https://discovery.nationalarchives.gov.uk/details/r/D7732547), comprising the top 2 rows, stopping at 36°N. 

#### Importing the required libraries

In [ ]:
%pip install -q folium
%pip install -q zipfile
import folium
import zipfile
import urllib.request

## Base map

The first step is to draw the base map, with a large amount of room around it to allow for context when overlaying the maps. 

In [ ]:
turkey_map = folium.Map(location=[39.9334, 32.8597], zoom_start=6)

turkey_map

## Overlaying maps

The `ImageOverlay` function is used to to place images on the map. There are a few parameters that need to be set: 
- `name` provides a label for the overlayed image, used in the layer control.
- `image` is the path to the image file - this can be local or a web address.
- `bounds` is the area that the image should cover. This is a list of the form `[[min/south_lat, min/west_lon], [max/north_lat, max/east_lon]]`.
- `opacity` is the transparency of the image, with 0 being fully transparent and 1 being fully opaque.

Setting the correct value for `bounds` can be tricky for two reasons: 
- The overlay map may be using a different [projection](https://en.wikipedia.org/wiki/Map_projection) to the base map (typically Web Mercator, as used in OpenStreetMap and Google Maps). If a key area is lined up correctly, a difference in map projection will mean that the rest of the map will drift out of alignment. 
- Older maps may be less accurate than modern maps, and may not line up perfectly with the base over their entire extent.

If overlays are proving hard to line up perfectly, the most appropriate solution is likely to be to line up at the area of key interest, and accept that the rest of the map will be out of alignment.

#### Adding the Discovery maps

In [ ]:
folium.raster_layers.ImageOverlay(
    name='Extract from "The future settlement of Eastern Turkey in Asia and Arabia. "',
    image="image_a.png",
    bounds=[[34,42],[37.5,48.5]],
    opacity=0.5,
    interactive=False,
    zindex=2,
).add_to(turkey_map)

folium.raster_layers.ImageOverlay(
    name='Extract from "Record Type: Memorandum Former Reference: GT 6506A Title: Maps"',
    image="image_b.png",
    bounds=[[35.5, 22],[45, 37]],
    opacity=0.5,
    interactive=False,
    zindex=2,
).add_to(turkey_map)    

turkey_map

#### Overlaying the Wikimedia map

In [ ]:
folium.raster_layers.ImageOverlay(
    name="Turkey internal administrative boundaries from Wikimedia",
    image="https://upload.wikimedia.org/wikipedia/commons/e/e5/Turkey_adm_location_map.svg",
    bounds=[[35.5, 25.4], [42.5, 45.0]],
    opacity=0.5,
    interactive=False,
    cross_origin=False,
    zindex=1,
).add_to(turkey_map)

turkey_map

## Other overlays

It is possible to overlay other information on the map, such as GeoJSON data. This can provide context, such as overlaying political boundaries over historical or topographical maps. Here, the World Boundaries GeoJSON data from The World Bank is used to provide details.  

In [ ]:
data = urllib.request.urlretrieve("https://datacatalogfiles.worldbank.org/ddh-published/0038272/DR0046666/wb_boundaries_geojson_highres.zip?versionId=2023-01-19T09:29:18.1778038Z", "wb_boundaries_geojson_highres.zip")
data_unzip = zipfile.ZipFile("wb_boundaries_geojson_highres.zip", "r")
data_unzip.extractall()

boundaries = "WB_Boundaries_GeoJSON_highres/WB_Admin0_boundary_lines.geojson"

folium.GeoJson(boundaries).add_to(turkey_map)

turkey_map

## Adding layer control

Having multiple layers present at the same time can result in it being hard to see the most relevant information. The `LayerControl` function provides a widget in the top right of the map, providing the option to toggle the visibility of these layers, allowing the user to show only the information relevant to them. The names used here are taken from the `name` parameters when the images were added. 

In [ ]:
folium.LayerControl().add_to(turkey_map)

turkey_map

## Conclusion

Overlaying multiple maps and images can provide invaluable context to historical research. This can be used to show how understandings of an area have changed over time, how political boundaries have shifted, or how the quality of maps has improved. It could also be used to provide additional context to maps of a small area, for example overlaying the troop positions within a specific battle on a map of the entire war. With the possibility to add many different layers and types of data, there are fields and tools that can be used to manage the data being displayed, making the interactive map useful to the user.